In [ ]:
pip install impyute

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/train.csv', encoding = 'cp949')


# 1.컬럼 확인

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.tail(5)

In [ ]:
test.isnull().sum()

#2.전처리

## 필요없는 컬럼 삭제

In [ ]:
train.drop("PassengerId", axis = 1, inplace = True)
test.drop("PassengerId", axis = 1, inplace = True)
train.drop("Name", axis = 1, inplace = True)
test.drop("Name", axis = 1, inplace = True)
train.drop("Ticket", axis = 1, inplace = True)
test.drop("Ticket", axis = 1, inplace = True)

In [ ]:
train.info()

## 편향성 확인

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['Age'], y = train['Survived'])
plt.ylabel('Survived', fontsize=13)
plt.xlabel('Age', fontsize=13)
plt.show()

In [ ]:
train[['Pclass']].boxplot()

In [ ]:
train[['Age']].boxplot()

In [ ]:
#Correlation map to see how features are correlated with SalePrice
import seaborn as sns

corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(data = corrmat, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

## 결측치 처리

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# train["Embarked"] = train["Embarked"].fillna("None")

In [ ]:
# del train['Ticket']
# del test['Ticket']

In [ ]:
train.head()

In [ ]:
# test["Fare"] = test["Fare"].fillna("None")

## 라벨인코더

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# train_result = le.fit_transform(train['Embarked'])
# test_result = le.fit_transform(test['Embarked'])

# # print(result)
# train['Embarked'] = train_result
# test['Embarked'] = test_result

## 다중대체법

In [ ]:
train_no_Survived = train.drop(['Survived'], axis=1)


In [ ]:
total = pd.concat([train_no_Survived,test],axis=0,ignore_index=True)


In [ ]:
import missingno as msno
msno.matrix(total)


In [ ]:
dummy = pd.get_dummies(total)

In [ ]:
dummy.shape

In [ ]:
total_impute = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy))
total_impute

In [ ]:
msno.matrix(total_impute)


In [ ]:
total_cols = list(dummy.columns)
total_impute.columns = total_cols

In [ ]:
X_train = total_impute[:891]
X_test = total_impute[891:]

In [ ]:
y_train = train['Survived']

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
pd.options.display.float_format = '{:.5f}'.format


#3.다중공선성

In [ ]:
Numeric_feature = [ col for col in train.columns if train[col].dtypes != "object"]
Numeric_feature

In [ ]:
y, X = dmatrices('Survived ~ Pclass+Age+SibSp+Parch+Fare', train, return_type = 'dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

#4.상관관계 분석

In [ ]:
import statsmodels.api as sm;
import scipy.stats as stats
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_use_Survived = train
dummy_two = pd.get_dummies(train_use_Survived)
total_impute2 = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy_two))
total_impute2

In [ ]:
total_cols2 = list(dummy_two.columns)
total_impute2.columns = total_cols2
total_impute2.head(20)

In [ ]:
corrmat = total_impute2.corr()
# plt.subplots(figsize=(12,9))
# sns.heatmap(data = corrmat, annot=True, 
# fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
abs(corrmat['Survived']).sort_values()

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
X_train = total_impute2[corrmat[abs(corrmat['Survived']).sort_values() > 0.02].index]
X_train = X_train.drop(['Survived'], axis=1)
y_train = train['Survived']

In [ ]:
# corr_with_d = lambda x: x.corrwith(train['Survived'])
# grouped = train.groupby('Survived')
# grouped.apply(corr_with_d) 

In [ ]:
category_feature = [ col for col in train.columns if train[col].dtypes == "object"]
train_cor = train[category_feature]
train_cor

#5.머신러닝

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=777)

##1)GaussianNB

In [ ]:
def GaussianNB_fuction(X_train,y_train):
    model = GaussianNB()
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X_test) # 예측치
    # y_true = y_test # 정답
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('GaussianNB rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##2)SVC

In [ ]:
def SVC_fuction(X_train,y_train):
    model = SVC()  # kernel = 'rbf'  : default
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X_test) # 예측치
    # y_true = y_test # 정답
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('SVC rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##3)RandomForestClassifier

In [ ]:
def RandomForestClassifier_fuction(X_train,y_train):
    model = RandomForestClassifier().fit(X=X_train, y=y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('RandomForestClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred))) 

##4)MLPClassifier

In [ ]:
def MLPClassifier_fuction(X_train,y_train):
    model = MLPClassifier().fit(X=X_train, y=y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('MLPClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##5)DecisionTreeClassifier

In [ ]:
def DecisionTreeClassifier_fuction(X_train,y_train):
    model = DecisionTreeClassifier().fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('DecisionTreeClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##6)KNeighborsClassifier

In [ ]:
def KNeighborsClassifier_fuction(X_train,y_train):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('KNeighborsClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##7)LinearDiscriminantAnalysis

In [ ]:
def LinearDiscriminantAnalysis_fuction(X_train,y_train):
    model = LinearDiscriminantAnalysis()
    model.fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('LinearDiscriminantAnalysis rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##8)LogisticRegression

In [ ]:
def LogisticRegression_fuction(X_train,y_train):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))
    print('Root Mean Squared Error:',rmse(y_pred,y_test))

##9)실행문

In [ ]:
GaussianNB_fuction(X_train,y_train)
SVC_fuction(X_train,y_train)
RandomForestClassifier_fuction(X_train,y_train)
MLPClassifier_fuction(X_train,y_train)
DecisionTreeClassifier_fuction(X_train,y_train)
KNeighborsClassifier_fuction(X_train,y_train)
LinearDiscriminantAnalysis_fuction(X_train,y_train)
LogisticRegression_fuction(X_train,y_train)

In [ ]:
GaussianNB_fuction(X_train,y_train)
SVC_fuction(X_train,y_train)
RandomForestClassifier_fuction(X_train,y_train)
MLPClassifier_fuction(X_train,y_train)
DecisionTreeClassifier_fuction(X_train,y_train)
KNeighborsClassifier_fuction(X_train,y_train)
LinearDiscriminantAnalysis_fuction(X_train,y_train)

#6.앙상블기법

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

# ensemble 할 model 정의
models = [
    # ('enc', ElasticNetCV()),
    # ('gnb',GaussianNB()),
    # ('svc',SVC()),
    ('rf',RandomForestClassifier()),
    ('mlpc',MLPClassifier()),
    ('dtc',DecisionTreeClassifier()),
    ('kc',KNeighborsClassifier()),
    # ('lda',LinearDiscriminantAnalysis()),
    ('lr',LogisticRegression())

]
voting_classifier = VotingClassifier(models, n_jobs=-1) #n_jobs 코어수
voting_classifier.fit(X_train, y_train)
voting_pred = voting_classifier.predict(X_test)
print('Root Mean Squared Error:',rmse(voting_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, voting_pred)))

In [ ]:
from sklearn.ensemble import StackingClassifier
models = [
    # ('enc', ElasticNetCV()),
    # ('gnb',GaussianNB()),
    # ('svc',SVC()),
    ('rf',RandomForestClassifier()),
    ('mlpc',MLPClassifier()),
    ('dtc',DecisionTreeClassifier()),
    ('kc',KNeighborsClassifier()),
    ('lda',LinearDiscriminantAnalysis()),
    # ('lr',LogisticRegression())

]
rf = RandomForestClassifier()
mlpc = MLPClassifier()
dtc = DecisionTreeClassifier()
kc = KNeighborsClassifier()
lda = LinearDiscriminantAnalysis()
lr = LogisticRegression()
stack_clf = StackingClassifier(models, final_estimator=mlpc, n_jobs=-1)
stack_clf.fit(X_train, y_train)
stack_pred = stack_clf.predict(X_test)
print('Root Mean Squared Error:',rmse(stack_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, stack_pred)))

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_regression
regr = BaggingClassifier(base_estimator=LinearDiscriminantAnalysis(),
                        n_estimators=10, random_state=0).fit(X_train, y_train)
bagging_pred = regr.predict(X_test)
print('Root Mean Squared Error:',rmse(bagging_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, bagging_pred)))